In [7]:
import json
from dataclasses import dataclass
from typing import Optional, Dict, Any, List

from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

@dataclass
class SearchAgentConfig:
    ollama_llm_model: str = 'deepseek-r1:8b'
    prompt_path: str = "C:/Users/PC/Desktop/Proyects/Proyectos/Bot/prompts/search_agent_prompt.txt"
    top_k: int = 6
    temperature: float = 0

class SearchAgent:
    def __init__(self, config: SearchAgentConfig):
        self.config = config
        
        # Carga del sistema de prompt optimizado
        try:
            with open(config.prompt_path, "r", encoding="utf-8") as f:
                system_content = f.read().strip()
        except FileNotFoundError:
            system_content = "Responde estrictamente en formato JSON."

        # Configuración del modelo con LangChain
        # Forzamos format="json" para que Ollama ayude al modelo a cumplir
        self.llm = ChatOllama(
            model=config.ollama_llm_model,
            temperature=config.temperature,
            format="json"
        )

        self.web_search = DuckDuckGoSearchResults(output_format="list")
        self.parser = JsonOutputParser()

        # Definimos el template para recibir la query y el JSON de evidencia
        self.prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_content),
            ("user", "Pregunta: {query}\nEvidencia disponible: {evidence_json}")
        ])

        # Cadena LCEL
        self.chain = self.prompt_template | self.llm | self.parser

    def search(self, query: str, k: Optional[int] = None) -> Dict[str, Any]:
        k = k or self.config.top_k

        # 1. Búsqueda
        search_results: List[Dict[str, Any]] = self.web_search.invoke({"query": query})
        
        # 2. Normalización con IDs (clave para el citado [1], [2]...)
        evidence = [
            {
                "id": i + 1,
                "source": r.get("link"),
                "title": r.get("title"),
                "excerpt": r.get("snippet")
            }
            for i, r in enumerate(search_results[:k])
        ]

        # 3. Invocación
        try:
            # Pasamos la evidencia como string JSON para que el prompt sea claro
            response = self.chain.invoke({
                "query": query, 
                "evidence_json": json.dumps(evidence, ensure_ascii=False)
            })
            return response
        except Exception as e:
            return {
                "query": query,
                "answer": "Error crítico al procesar la respuesta del modelo.",
                "evidence": evidence,
                "notes": [f"Detalle técnico: {str(e)}"]
            }


In [10]:
# Ejecución de prueba
if __name__ == "__main__":
    agent = SearchAgent(SearchAgentConfig())
    # Probamos con un tema de actualidad para forzar el uso de evidencia
    print(agent.search("Protestas en Iran - Enero 2026"))

{'query': 'Protestas en Iran - Enero 2026', 'answer': 'Las protestas en Irán comenzaron a finales de 2025 y se extendieron por todo el país en enero de 2026 [1, 3]. Han persistido por más de 20 días, con una ola de descontento que involucra a todas las 31 provincias [1]. La represión por parte de las fuerzas policiales ha resultado en al menos 28 víctimas mortales entre el 31 de diciembre de 2025 y el 3 de enero de 2026 [1]. Manifestaciones también tuvieron eco internacional, como en Londres, donde se exigieron cambios políticos [4]. El Secretario General de la ONU expresó consternación por los informes de violencia y uso excesivo de la fuerza [1].', 'evidence': [{'id': 1, 'source': 'https://cnnespanol.cnn.com/2026/01/13/mundo/iran-protestas-masivas-eeuu-implicarse-trax', 'title': '¿Por qué hay protestas masivas en Irán y podría involucrarse ... EN VIVO | A 20 días del inicio de las protestas, el número de ... Cronología visual de las protestas en Irán: cómo empezaron y ... Protestas e